This notebook is part of the `deepcell-tf` documentation: https://deepcell.readthedocs.io/.

# Nuclear segmentation and tracking

In [1]:
import os

import numpy as np

import imageio
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import backend as K

## Prepare nuclear data

Use `imageio` to load each frame of a gif to form a test dataset.

In [2]:
from deepcell.datasets.tracked import hela_s3

(X_train, y_train), (X_test, y_test) = hela_s3.load_tracked_data(seed=0)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

6370648064/6370641920 [==============================] - 239s 0us/step
X_train shape: (144, 40, 216, 256, 1)
X_test shape: (36, 40, 216, 256, 1)


In [3]:
x = X_train[56]  # chosen batch with divisions

In [4]:
def plot(im):
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.imshow(im)
    plt.axis('off')
    plt.title('Raw Image Data')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    plt.close(fig)

    return image

imageio.mimsave('raw.gif', [plot(x[i, ..., 0]) for i in range(x.shape[0])])

### View .GIF of raw cells

![Raw Gif](./raw.gif)

## Nuclear Segmentation

### Initialize nuclear model

The application will download pretrained weights for nuclear segmentation. For more information about application objects, please see our [documentation](https://deepcell.readthedocs.io/en/master/API/deepcell.applications.html).

In [5]:
from deepcell.applications import NuclearSegmentation

app = NuclearSegmentation()

95191040/95187971 [==============================] - 7s 0us/step


## Use the application to generate labeled images

Typically, neural networks perform best on test data that is similar to the training data. In the realm of biological imaging, the most common difference between datasets is the resolution of the data measured in microns per pixel. The training resolution of the model can be identified using `app.model_mpp`.

In [6]:
print('Training Resolution:', app.model_mpp, 'microns per pixel')

Training Resolution: 0.65 microns per pixel


The resolution of the input data can be specified in `app.predict` using the `image_mpp` option. The `Application` will rescale the input data to match the training resolution and then rescale to the original size before returning the labeled image.

In [7]:
y_pred = app.predict(x, image_mpp=0.65)

print(y_pred.shape)

(40, 216, 256, 1)


### Save labeled images as a gif to visualize

In [8]:
def plot(im1, im2, vmin, vmax):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(im1)
    ax[0].axis('off')
    ax[0].set_title('Raw')
    ax[1].imshow(im2, cmap='jet', vmin=vmin, vmax=vmax)
    ax[1].set_title('Segmented')
    ax[1].axis('off')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return image

imageio.mimsave(
    './labeled.gif',
    [plot(x[i,...,0], y_pred[i,...,0], y_pred.min(), y_pred.max())
     for i in range(y_pred.shape[0])]
)

### View .GIF of segmented cells

The `NuclearSegmentation` application was able to create a label mask for every cell in every frame!

![Segmented GIF](./labeled.gif)

## Cell Tracking

The `NuclearSegmentation` worked well, but the cell labels of the same cell are not preserved across frames. To resolve this problem, we can use the `CellTracker`! This object will use another `CellTrackingModel` to compare all cells and determine which cells are the same across frames, as well as if a cell split into daughter cells.

### Initalize CellTracking application

Create an instance of `deepcell.applications.CellTracking`.

In [9]:
from deepcell.applications import CellTracking

tracker = CellTracking()

647168/645608 [==============================] - 0s 0us/step


### Track the cells

In [10]:
tracked_data = tracker.track(np.copy(x), y_pred)

### Visualize tracking results

In [11]:
# Convert tracking results to a dictionary

X = tracked_data['X']  # raw X data
y = tracked_data['y_tracked']  # tracked y data

In [12]:
def plot(im1,im2,vmin,vmax):
    fig, ax = plt.subplots(1, 2, figsize=(12, 6))
    ax[0].imshow(im1)
    ax[0].axis('off')
    ax[0].set_title('Raw')
    ax[1].imshow(im2, cmap='jet', vmin=vmin, vmax=vmax)
    ax[1].set_title('Tracked')
    ax[1].axis('off')

    fig.canvas.draw()  # draw the canvas, cache the renderer
    image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
    image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
    plt.close(fig)

    return image

imageio.mimsave('tracks.gif', [plot(X[i,...,0], y[i,...,0], y.min(), y.max())
                               for i in range(y_pred.shape[0])])

### View .GIF of tracked cells

Now that we've finished using `CellTracker.track_cells`, not only do the annotations preserve label across frames, but the lineage information has been saved in `CellTracker.tracks`.

![Tracked Cells GIF](./tracks.gif)